In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import numpy as np

In [ ]:
!pip install tensorflow==2.2.0rc2

In [27]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [3]:
num_class = 5
def creat_compiled_model():
    model = Sequential()
    model.add(NASNetMobile(input_shape=(224, 224, 3), include_top=False,
                           weights='imagenet', pooling='avg'))
    model.add(Dense(num_class, activation='softmax'))
    model.layers[0].trainable = True
    model.layers[1].trainable = True
    sgd = SGD(learning_rate=0.01, momentum=0.01, nesterov=False)
    model.compile(optimizer=sgd, loss='hinge',  metrics=['accuracy'])
    return model

In [4]:
model3 = creat_compiled_model()

In [5]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit_generator
train_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)

In [7]:
train_generator = train_datagen.flow_from_directory('labelled_data', batch_size=4)
model3.fit(train_generator, epochs=3, steps_per_epoch=3)

Found 99 images belonging to 5 classes.
  ...
    to  
  ['...']
Train for 3 steps
Epoch 1/3
3/3 [==============================] - 36s 12s/step - loss: 1.1261 - accuracy: 0.1667
Epoch 2/3
3/3 [==============================] - 4s 1s/step - loss: 1.1180 - accuracy: 0.2500
Epoch 3/3
3/3 [==============================] - 4s 1s/step - loss: 1.0644 - accuracy: 0.4167


In [8]:
#del image
image = load_img('./labelled_data/Front/00035.jpg', target_size=(224, 224))
#print(image)
image = img_to_array(image)
#print(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
#print(image)
# cette ligne a verifier le self?
probs = model3.predict(image)[0]
pred_labels, pred_scores = [], []
print(probs)
list_of_labels = ["Back", "Discard", "Front", "Left", "Right"]
label_idx = np.argmax(probs)
label = list_of_labels[label_idx]
score = probs[label_idx]
pred_labels.append(label)
pred_scores.append(score.item())
print(pred_labels)
print(pred_scores)

[9.9915600e-01 5.7631783e-04 6.7334477e-05 1.7033714e-04 3.0036743e-05]
['Back']
[0.9991559982299805]


In [9]:
image = load_img('./labelled_data/Right/07944.jpg', target_size=(224, 224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
print(model3.predict_classes(image)[0])
x = int(model3.predict_classes(image)[0])
print(x, type(x))
y = 5
print(y, type(y))

0
0 <class 'int'>
5 <class 'int'>


In [10]:
sgd = SGD(learning_rate=0.01, momentum=0.01, nesterov=False)
model3.compile(optimizer=sgd, loss='hinge', metrics=['accuracy'])

In [11]:
model3.save("./labelled_data/trained_model", save_format='h5')

In [12]:
loaded_model = load_model("./labelled_data/trained_model")